In [15]:
import torch

In [16]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

X, Y = datasets.load_iris(return_X_y=True)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, stratify=Y, random_state=123)

X_train, X_test, Y_train, Y_test = torch.tensor(X_train, dtype=torch.float32),torch.tensor(X_test, dtype=torch.float32),torch.tensor(Y_train, dtype=torch.long),torch.tensor(Y_test, dtype=torch.long)

samples, features = X_train.shape
classes = Y_test.unique()
print(features)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

4
torch.Size([120, 4]) torch.Size([30, 4]) torch.Size([120]) torch.Size([30])


In [17]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean)/ std
X_test = (X_test - mean)/ std

In [18]:
from torch import nn

class PercepClassifier(nn.Module):
    def __init__(self):
        super(PercepClassifier, self).__init__()
        self.first_layer = nn.Linear(features, 5)
        self.second_layer = nn.Linear(5, 10)
        self.third_layer = nn.Linear(10, 15)
        self.fourth_layer = nn.Linear(15, 20)
        self.final_layer = nn.Linear(20,3)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X_batch):
        layer_out = self.relu(self.first_layer(X_batch))
        layer_out = self.relu(self.second_layer(layer_out))
        layer_out = self.relu(self.third_layer(layer_out))
        layer_out = self.relu(self.fourth_layer(layer_out))

        return self.softmax(self.final_layer(layer_out))

In [19]:
classifier = PercepClassifier()
preds = classifier(X_train[:5])
preds

tensor([[0.2656, 0.3754, 0.3590],
        [0.2644, 0.3744, 0.3612],
        [0.2603, 0.3772, 0.3626],
        [0.2673, 0.3761, 0.3566],
        [0.2664, 0.3754, 0.3582]], grad_fn=<SoftmaxBackward0>)

In [20]:
def TrainModel(model, loss_func, optimizer, X, Y, epochs=500):
    for i in range(epochs):
        preds = model(X)
        loss = loss_func(preds, Y)
        optimizer.zero_grad() 
        optimizer.step() 
        if i % 100 == 0: 
            print("NegLogLoss : {:.2f}".format(loss))

In [21]:
from torch.optim import Adamax
torch.manual_seed(42)
epochs = 1500
learning_rate = torch.tensor(1/1e2) # 0.01

classifier = PercepClassifier()
nll_loss = nn.NLLLoss()
optimizer = Adamax(params=classifier.parameters(), lr=learning_rate)

TrainModel(classifier, nll_loss, optimizer, X_train, Y_train, epochs=epochs)

NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33
NegLogLoss : -0.33


In [22]:
test_preds = classifier(X_test) 

test_preds = torch.argmax(test_preds, axis=1) 

train_preds = classifier(X_train)

train_preds = torch.argmax(train_preds, axis=1) 

test_preds[:5], train_preds[:5]

(tensor([2, 2, 2, 2, 2]), tensor([2, 2, 2, 2, 2]))

In [23]:
from sklearn.metrics import accuracy_score

print("Train Accuracy : {:.2f}".format(accuracy_score(Y_train, train_preds)))
print("Test  Accuracy : {:.2f}".format(accuracy_score(Y_test, test_preds)))

Train Accuracy : 0.33
Test  Accuracy : 0.33


In [24]:
from sklearn.metrics import classification_report

print("Test Data Classification Report : ")
print(classification_report(Y_test, test_preds))

Test Data Classification Report : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        10
           2       0.33      1.00      0.50        10

    accuracy                           0.33        30
   macro avg       0.11      0.33      0.17        30
weighted avg       0.11      0.33      0.17        30



c:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
